we'll take the data from api according to the instruments we've gotten before
convert them into dataframe and save them for furthure use

In [1]:
import requests
import authentication
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle('./data/instruments.pkl')

In [4]:
our_currencies = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [5]:
a = (ins_df.name.unique())
print(a)
print(len(a))

['AUD_USD' 'GBP_NZD' 'CHF_ZAR' 'USD_CNH' 'NZD_USD' 'NZD_SGD' 'USD_NOK'
 'NZD_JPY' 'EUR_HUF' 'USD_CAD' 'ZAR_JPY' 'GBP_ZAR' 'SGD_JPY' 'USD_JPY'
 'EUR_TRY' 'EUR_JPY' 'AUD_SGD' 'EUR_NZD' 'GBP_HKD' 'CHF_JPY' 'EUR_HKD'
 'GBP_CAD' 'USD_THB' 'GBP_CHF' 'NZD_CHF' 'AUD_HKD' 'USD_CHF' 'SGD_CHF'
 'USD_TRY' 'GBP_JPY' 'USD_PLN' 'GBP_AUD' 'HKD_JPY' 'AUD_CHF' 'EUR_SEK'
 'USD_SGD' 'TRY_JPY' 'EUR_AUD' 'CHF_HKD' 'EUR_SGD' 'NZD_CAD' 'CAD_JPY'
 'USD_CZK' 'USD_ZAR' 'USD_DKK' 'EUR_CHF' 'AUD_JPY' 'USD_HUF' 'EUR_CAD'
 'EUR_ZAR' 'EUR_USD' 'GBP_USD' 'USD_MXN' 'NZD_HKD' 'USD_HKD' 'CAD_SGD'
 'AUD_CAD' 'EUR_PLN' 'GBP_PLN' 'EUR_NOK' 'CAD_HKD' 'EUR_GBP' 'CAD_CHF'
 'USD_SEK' 'GBP_SGD' 'EUR_DKK' 'AUD_NZD' 'EUR_CZK']
68


In [6]:
def fetch_candles(pair_name, count, granularity):
    url = f"{authentication.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=authentication.SECURE_HEADER)
    return response.status_code, response.json()

In [7]:
code, res = fetch_candles("EUR_USD", '20', 'H1')
print(code)
print(res)

200
{'instrument': 'EUR_USD', 'granularity': 'H1', 'candles': [{'complete': True, 'volume': 5693, 'time': '2022-11-16T20:00:00.000000000Z', 'bid': {'o': '1.03914', 'h': '1.04040', 'l': '1.03822', 'c': '1.03970'}, 'mid': {'o': '1.03920', 'h': '1.04048', 'l': '1.03830', 'c': '1.03978'}, 'ask': {'o': '1.03927', 'h': '1.04056', 'l': '1.03838', 'c': '1.03986'}}, {'complete': True, 'volume': 2813, 'time': '2022-11-16T21:00:00.000000000Z', 'bid': {'o': '1.03970', 'h': '1.03984', 'l': '1.03881', 'c': '1.03934'}, 'mid': {'o': '1.03978', 'h': '1.03991', 'l': '1.03893', 'c': '1.03944'}, 'ask': {'o': '1.03987', 'h': '1.04000', 'l': '1.03903', 'c': '1.03954'}}, {'complete': True, 'volume': 4079, 'time': '2022-11-16T22:00:00.000000000Z', 'bid': {'o': '1.03885', 'h': '1.03953', 'l': '1.03856', 'c': '1.03925'}, 'mid': {'o': '1.03920', 'h': '1.03994', 'l': '1.03906', 'c': '1.03942'}, 'ask': {'o': '1.03954', 'h': '1.04044', 'l': '1.03948', 'c': '1.03959'}}, {'complete': True, 'volume': 2284, 'time': '20

In [8]:
def get_candles_df(json_response):

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [9]:
df = get_candles_df(res)

In [10]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-11-16T20:00:00.000000000Z,5693,1.03920,1.04048,1.03830,1.03978,1.03914,1.04040,1.03822,1.03970,1.03927,1.04056,1.03838,1.03986
1,2022-11-16T21:00:00.000000000Z,2813,1.03978,1.03991,1.03893,1.03944,1.03970,1.03984,1.03881,1.03934,1.03987,1.04000,1.03903,1.03954
2,2022-11-16T22:00:00.000000000Z,4079,1.03920,1.03994,1.03906,1.03942,1.03885,1.03953,1.03856,1.03925,1.03954,1.04044,1.03948,1.03959
3,2022-11-16T23:00:00.000000000Z,2284,1.03950,1.03985,1.03916,1.03936,1.03933,1.03975,1.03906,1.03927,1.03968,1.03995,1.03924,1.03944
4,2022-11-17T00:00:00.000000000Z,3220,1.03936,1.03952,1.03856,1.03866,1.03928,1.03943,1.03848,1.03857,1.03944,1.03962,1.03863,1.03875


In [11]:
def save_file(candle_df, pair, granularity):
    candle_df.to_pickle(f'historical_data/{pair}_{granularity}.pkl')

In [12]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, 'Error 200')
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [13]:
for p1 in our_currencies:
    for p2 in our_currencies:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")

EUR_USD loaded 3999 candles from 2022-03-29T18:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2022-03-29T16:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2022-03-29T10:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2022-03-29T19:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2022-03-29T18:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2022-03-29T19:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
USD_JPY loaded 3999 candles from 2022-03-29T18:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
USD_CHF loaded 3999 candles from 2022-03-29T14:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
USD_CAD loaded 3999 candles from 2022-03-29T19:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
GBP_USD loaded 3999 candles from 2022-03-29T18:00:00.000000000Z to 2022-11-17T14:00:00.000000000Z
GBP_JPY loaded 3999 